# Multi-Label Image Classification Model 

This note is depnds on the original taturial from this [link](https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/)

## ReadingDataSet

this porject used IMDB dataset which content the meta data and the posters of the moives and tv-shows. 
You can download the data from this [link](https://www.cs.ccu.edu.tw/~wtchu/projects/MoviePoster/index.html) 

In [11]:
import os
from os.path import join as pjoin
from bson.json_util import loads, dumps
import pandas as pd
from pandas import json_normalize
import numpy as np


main_folder="/home/zaher/DataFolder/Movie_Poster_Dataset"
csv_file="IMDB_DB.csv"
images="images"


df=pd.read_csv(pjoin(main_folder,csv_file))


genre=[]
genre_cats=[]
genre_cats_dict=dict()
counter=0
for x,i in df.iterrows():
    gs=[]
    gs_cat=[]
    
    for g in str(i[5]).split(','):
        s=g
        s=s.lstrip()
        s=s.rstrip()
        if s== 'nan':
            gs=None
            gs_cat=None
            break
        if not s in genre_cats_dict.keys():
            genre_cats_dict[s]=counter
            counter+=1
        
        gs_cat.append(genre_cats_dict[s])
        gs.append(s)
        
    genre.append(gs)
    genre_cats.append(gs_cat)
    

df['Genres']=genre
df['Genres_cat']=genre_cats

df=df.dropna(subset=['Genres_cat'])

### Create training, val, & testing data

In [12]:
from sklearn.model_selection import train_test_split

split_ratio=30

split=int(len(df)*split_ratio/100)

X_train, X_test = df[split:],df[:split]

len(X_test),len(X_train)

(2650, 6186)

In [13]:
X_train.head()

,Released,Writer,Type,imdbVotes,Response,Genre,Metascore,imdbID,imdbRating,Title,...,Language,Country,Box_office,Runtime,Actors,Director,Awards,Poster,Genres,Genres_cat
2662,NaN,T. Lee Beideck,movie,65,True,Comedy,NaN,tt0385666,6.9,Drivers Wanted,...,English,USA,"19,875",94 min,"David Sirk, Brian Osborne, Dave Spiecher, Sal ...",T. Lee Beideck,NaN,http://ia.media-imdb.com/images/M/MV5BMTU1MDYw...,[Comedy],[3]
2663,30 Jun 2007,"Lisa Lax, Elizabeth Massie, Nancy Stern",movie,167,True,Documentary,61.0,tt0447016,7.7,Emmanuel's Gift,...,English,USA,"23,578",80 min,"Robin Williams, Oprah Winfrey","Lisa Lax, Nancy Stern",NaN,http://ia.media-imdb.com/images/M/MV5BMjA1ODIw...,[Documentary],[10]
2664,04 Feb 2005,Michael Sandager,movie,35,True,Documentary,NaN,tt0447658,5.0,Letters to Dear Wendy,...,"Danish, English",Denmark,NaN,25 min,"Michael Angarano, Jamie Bell, Danso Gordon, Je...",Michael Sandager,NaN,NaN,[Documentary],[10]
2665,NaN,NaN,movie,NaN,True,Adult,NaN,tt0495122,NaN,Hot Slutty Summer,...,NaN,NaN,NaN,NaN,"Skyla Banks, Jezebelle Bond, Lena Juliette, Sh...",NaN,NaN,NaN,[Adult],[22]
2666,18 Feb 2005,Rodney Patrick Vaccaro,movie,"1,263",True,"Comedy, Drama, Romance",21.0,tt0363504,6.1,Bigger Than the Sky,...,English,USA,"20,149",106 min,"Marcus Thomas, John Corbett, Amy Smart, Sean A...",Al Corley,NaN,http://ia.media-imdb.com/images/M/MV5BMTIzNjE2...,"[Comedy, Drama, Romance]","[3, 7, 11]"


In [14]:
X_train_img_id,X_train_genres=X_train['imdbID'],X_train['Genres_cat']
X_test_img_id,X_test_genres=X_test['imdbID'],X_test['Genres_cat']

In [15]:
genre_cats_dict

{'Action': 0,
 'Adventure': 1,
 'Animation': 2,
 'Comedy': 3,
 'Sci-Fi': 4,
 'Fantasy': 5,
 'Family': 6,
 'Drama': 7,
 'Thriller': 8,
 'Mystery': 9,
 'Documentary': 10,
 'Romance': 11,
 'Crime': 12,
 'Biography': 13,
 'Sport': 14,
 'Horror': 15,
 'Music': 16,
 'Musical': 17,
 'Short': 18,
 'History': 19,
 'War': 20,
 'Western': 21,
 'Adult': 22,
 'Reality-TV': 23,
 'News': 24,
 'Talk-Show': 25,
 'Game-Show': 26}

In [16]:
from util import DataGenerator
import tensorflow as tf
import tensorflow.keras as k

params = {'dim': (255,345),
          'batch_size': 64,
          'n_class': 6,
          'n_channel': 1,
          'shuffle': True}

train_gen=DataGenerator(X_train_img_id.values,X_train_genres.values,**params)
test_gen=DataGenerator(X_test_img_id.values,X_test_genres.values,**params)

l_input = k.layers.Input(shape=(255,345,3))
xinput=k.layers.Conv2D(filters=32,kernel_size=3)(l_input)
flat_l = k.layers.Flatten()(xinput)
x=tf.keras.layers.Dense(8,activation='relu')(flat_l)


model=k.Model(inputs=l_input,outputs=x)
model.compile(tf.optimizers.RMSprop(0.001), loss='mse')

model.fit(x=train_gen,                             use_multiprocessing=True,  workers=6)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [17]:
def __get_label(genre):
    y=np.zeros((10))
    for i in genre:
        y[i]=1
    return y

In [19]:
__get_label([3])

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [21]:
for i in X_train_genres.values:
    print(i)

[3]
[10]
[10]
[22]
[3, 7, 11]
[10]
[10, 13, 16]
[10]
[3]
[3, 7, 11]
[18, 3]
[10, 13]
[18]
[3, 11]
[12, 7, 9]
[10, 18]
[8]
[3, 7]
[3, 7, 16]
[3, 11]
[22]
[3, 7]
[0, 12, 7]
[3, 7, 19]
[10, 20]
[24]
[10]
[7]
[10]
[3, 7, 16]
[15, 8]
[3]
[10]
[10]
[10]
[18, 7]
[0, 1, 3]
[3, 11]
[3, 12, 7]
[7]
[0, 7]
[10]
[11, 3, 7]
[7]
[3, 10]
[15, 8]
[0, 3, 12]
[1, 3, 5]
[0, 1]
[3, 5, 15]
[0, 7, 6]
[3]
[7, 16, 11]
[0, 1, 3]
[0, 1, 4]
[3, 5, 11]
[7, 16, 17]
[13, 7, 16]
[9, 8]
[7, 14]
[1, 7]
[3]
[9, 4, 8]
[3, 7, 16]
[3]
[0, 4]
[0, 3, 12]
[3, 5, 11]
[7]
[7, 19, 20]
[0, 1, 5]
[0, 1, 4]
[1, 7, 11]
[0, 1, 4]
[0, 3]
[3, 7]
[1, 7, 19]
[3]
[12, 7, 16]
[3, 7, 6]
[15]
[3, 11]
[3, 7, 11]
[3, 7, 11]
[3, 11]
[0, 1, 7]
[12, 7, 9]
[0, 1, 12]
[3, 5]
[3, 16]
[3]
[3]
[3, 11, 14]
[1, 7, 6]
[3, 20]
[3, 11]
[0, 12, 7]
[0, 12, 8]
[0, 3, 12]
[0, 15, 4]
[7, 16]
[3, 7, 17]
[15]
[3, 15, 4]
[0, 1, 5]
[0, 1, 3]
[0, 7, 8]
[3, 7, 11]
[1, 4, 8]
[3]
[7]
[0, 7, 3]
[7, 11]
[7, 16]
[3, 7]
[3]
[7, 8, 12]
[15, 10, 8]
[7]
[3, 11]
[12, 9, 8]
[3,